# Santander benchmark - ML model training with xgboost
## `https://www.kaggle.com/c/santander-customer-transaction-prediction/overview`

### The goal is to measure the total execution time of flytekit workflow:
* ### [Workflow execution cell - 2 task division](#execution_cell_1)

* ### [Workflow execution cell - more detalized task division](#execution_cell_2)

In [1]:
import warnings
import time
from timeit import default_timer as timer

warnings.filterwarnings("ignore")

In [2]:
import logging
from flytekit.loggers import logger

logger.setLevel(level=logging.WARN)
logger.getEffectiveLevel

<bound method Logger.getEffectiveLevel of <Logger flytekit (WARNING)>>

In [3]:
from collections import OrderedDict
from functools import partial
from timeit import default_timer as timer
from dataclasses import dataclass
import typing
from flytekit import Resources, task, workflow, dynamic
from flytekit.types.file import FlyteFile
from flytekit.types.schema import FlyteSchema
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost

### Common part: global variables

In [4]:
DATASET_PATH = "https://modin-datasets.s3.amazonaws.com/santander/train.csv"  # full path is to write
ETL_KEYS = ["t_readcsv", "t_etl", "t_connect"]
ML_KEYS = ["t_train_test_split", "t_dmatrix", "t_training", "t_infer", "t_ml"]
ML_SCORE_KEYS = ["mse_mean", "cod_mean", "mse_dev"]

VAR_COLS = ["var_%s" % i for i in range(200)]
COLUMNS_NAMES = ["ID_code", "target"] + VAR_COLS
COLUMNS_TYPES = ["object", "int64"] + ["float64" for _ in range(200)]

### Workflow consisting from 2 tasks

In [6]:
# from utils

def load_data_pandas(
    filename,
    columns_names=None,
    columns_types=None,
    header=None,
    nrows=None,
    use_gzip=False,
    parse_dates=None,
):
    types = None
    if columns_types:
        types = {columns_names[i]: columns_types[i] for i in range(len(columns_names))}
    return pd.read_csv(
        filename,
        names=columns_names,
        nrows=nrows,
        header=header,
        dtype=types,
        compression="gzip" if use_gzip else None,
        parse_dates=parse_dates,
    )

In [7]:
def mse(y_test, y_pred):
    return ((y_test - y_pred) ** 2).mean()


def cod(y_test, y_pred):
    y_bar = y_test.mean()
    total = ((y_test - y_bar) ** 2).sum()
    residuals = ((y_test - y_pred) ** 2).sum()
    return 1 - (residuals / total)

In [8]:
def split_step(data, target):
    t0 = timer()
    train, valid = data[:-10000], data[-10000:]
    split_time = timer() - t0

    x_train = train.drop([target], axis=1)

    y_train = train[target]

    x_test = valid.drop([target], axis=1)

    y_test = valid[target]

    return (x_train, y_train, x_test, y_test), split_time

In [9]:
@task
def etl_pandas(
    filename: str,
    columns_names: typing.List[str],
    columns_types: typing.List[str],
    etl_keys: typing.List[str],
) -> (pd.DataFrame, typing.Dict[str, float]):
    etl_times = {key: 0.0 for key in etl_keys}

    t0 = timer()
    train_pd = load_data_pandas(
        filename=filename,
        columns_names=columns_names,
        columns_types=columns_types,
        header=0,
        use_gzip=filename.endswith(".gz"),
    )
    etl_times["t_readcsv"] = timer() - t0

    t_etl_begin = timer()

    for i in range(200):
        col = "var_%d" % i
        var_count = train_pd.groupby(col).agg({col: "count"})

        var_count.columns = ["%s_count" % col]
        var_count = var_count.reset_index()

        train_pd = train_pd.merge(var_count, on=col, how="left")

    for i in range(200):
        col = "var_%d" % i

        mask = train_pd["%s_count" % col] > 1
        train_pd.loc[mask, "%s_gt1" % col] = train_pd.loc[mask, col]

    train_pd = train_pd.drop(["ID_code"], axis=1)
    etl_times["t_etl"] = timer() - t_etl_begin

    return train_pd, etl_times

In [10]:
@task
def ml(
    ml_data: pd.DataFrame, target: str, ml_keys: typing.List[str], ml_score_keys: typing.List[str]
) -> (typing.Dict[str, float], typing.Dict[str, float]):

    ml_times = {key: 0.0 for key in ml_keys}
    ml_scores = {key: 0.0 for key in ml_score_keys}

    (x_train, y_train, x_test, y_test), ml_times["t_train_test_split"] = split_step(
        ml_data, target
    )

    t0 = timer()
    training_dmat_part = xgboost.DMatrix(data=x_train, label=y_train)
    testing_dmat_part = xgboost.DMatrix(data=x_test, label=y_test)
    ml_times["t_dmatrix"] = timer() - t0

    watchlist = [(testing_dmat_part, "eval"), (training_dmat_part, "train")]
    #     hard_code: cpu_params cannot be an input, cause values are not homogeneous
    xgb_params = {
        "objective": "binary:logistic",
        "tree_method": "hist",
        "max_depth": 1,
        "nthread": 56,
        "eta": 0.1,
        "silent": 1,
        "subsample": 0.5,
        "colsample_bytree": 0.05,
        "eval_metric": "auc",
    }

    t0 = timer()
    model = xgboost.train(
        xgb_params,
        dtrain=training_dmat_part,
        num_boost_round=10000,
        evals=watchlist,
        early_stopping_rounds=30,
        maximize=True,
        verbose_eval=1000,
    )
    ml_times["t_train"] = timer() - t0

    t0 = timer()
    yp = model.predict(testing_dmat_part)
    ml_times["t_inference"] = timer() - t0

    ml_scores["mse"] = mse(y_test, yp)
    ml_scores["cod"] = cod(y_test, yp)

    ml_times["t_ml"] += ml_times["t_train"] + ml_times["t_inference"]

    return ml_scores, ml_times

In [11]:
@workflow
def santander_ml_wf(
    filename: str = DATASET_PATH,
    columns_names: typing.List[str] = COLUMNS_NAMES,
    columns_types: typing.List[str] = COLUMNS_TYPES,
    etl_keys: typing.List[str] = ETL_KEYS,
    target: str = "target",
    ml_keys: typing.List[str] = ML_KEYS,
    ml_score_keys: typing.List[str] = ML_SCORE_KEYS,
) -> (typing.Dict[str, float], typing.Dict[str, float]):
    df, etl_times = etl_pandas(
        filename=filename,
        columns_names=columns_names,
        columns_types=columns_types,
        etl_keys=etl_keys,
    )
    return ml(ml_data=df, target=target, ml_keys=ml_keys, ml_score_keys=ml_score_keys)

## <a id='execution_cell_1'>Workflow execution - 2 task division</a>

In [12]:
%%time

santander_ml_wf()

[17:25:35] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-auc:0.53415	train-auc:0.52877
[1000]	eval-auc:0.89019	train-auc:0.90135
[2000]	eval-auc:0.90305	train-auc:0.91705
[3000]	eval-auc:0.90846	train-auc:0.92279
[4000]	eval-auc:0.91038	train-auc:0.92592
[4347]	eval-auc:0.91130	train-auc:0.92684
CPU times: user 2h 1min 57s, sys: 56.6 s, total: 2h 2min 54s
Wall time: 4min 13s


DefaultNamedTupleOutput(o0={'mse_mean': 0.0, 'cod_mean': 0.0, 'mse_dev': 0.0, 'mse': 0.054204980425112737, 'cod': 0.40511252848379153}, o1={'t_train_test_split': 9.94708389043808e-05, 't_dmatrix': 1.2653189827688038, 't_training': 0.0, 't_infer': 0.0, 't_ml': 136.55974651966244, 't_train': 136.54124352429062, 't_inference': 0.018502995371818542})

## Workflow consisting from more detalized tasks

In [5]:
@task
def load_data_pandas(
    filename: str,
    columns_names: typing.List[str],
    columns_types: typing.List[str],
    use_gzip: bool = False,
) -> pd.DataFrame:
    types = None
    if columns_types:
        types = {columns_names[i]: columns_types[i] for i in range(len(columns_names))}
    return pd.read_csv(
        filename,
        names=columns_names,
        dtype=types,
        header=0,
        compression="gzip" if use_gzip else None,
    )

In [6]:
@task
def mse(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    return ((y_test - y_pred) ** 2).mean()


@task
def cod(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    y_bar = y_test.mean()
    total = ((y_test - y_bar) ** 2).sum()
    residuals = ((y_test - y_pred) ** 2).sum()
    return 1 - (residuals / total)

In [7]:
@task
def split_step(
    data: pd.DataFrame, target: str
) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, float):
    t0 = timer()
    train, valid = data[:-10000], data[-10000:]
    split_time = timer() - t0

    x_train = train.drop([target], axis=1)

    y_train = train[target]

    x_test = valid.drop([target], axis=1)

    y_test = valid[target]

    return x_train, pd.DataFrame(y_train), x_test, pd.DataFrame(y_test), split_time

In [8]:
@dynamic
def etl_pandas(
    filename: str,
    columns_names: typing.List[str],
    columns_types: typing.List[str],
    etl_keys: typing.List[str],
) -> (pd.DataFrame, typing.Dict[str, float]):
    etl_times = {key: 0.0 for key in etl_keys}

    t0 = timer()
    train_pd = load_data_pandas(
        filename=filename,
        columns_names=columns_names,
        columns_types=columns_types,
        use_gzip=filename.endswith(".gz"),
    )
    etl_times["t_readcsv"] = timer() - t0

    t_etl_begin = timer()

    for i in range(200):
        col = "var_%d" % i
        var_count = train_pd.groupby(col).agg({col: "count"})

        var_count.columns = ["%s_count" % col]
        var_count = var_count.reset_index()

        train_pd = train_pd.merge(var_count, on=col, how="left")

    for i in range(200):
        col = "var_%d" % i

        mask = train_pd["%s_count" % col] > 1
        train_pd.loc[mask, "%s_gt1" % col] = train_pd.loc[mask, col]

    train_pd = train_pd.drop(["ID_code"], axis=1)
    etl_times["t_etl"] = timer() - t_etl_begin

    return train_pd, etl_times

In [9]:
@dynamic
def ml(
    ml_data: pd.DataFrame, target: str, ml_keys: typing.List[str], ml_score_keys: typing.List[str]
) -> (typing.Dict[str, float], typing.Dict[str, float]):

    ml_times = {key: 0.0 for key in ml_keys}
    ml_scores = {key: 0.0 for key in ml_score_keys}

    x_train, y_train, x_test, y_test, ml_times["t_train_test_split"] = split_step(
        data=ml_data, target=target
    )

    t0 = timer()
    training_dmat_part = xgboost.DMatrix(data=x_train, label=y_train)
    testing_dmat_part = xgboost.DMatrix(data=x_test, label=y_test)
    ml_times["t_dmatrix"] = timer() - t0

    watchlist = [(testing_dmat_part, "eval"), (training_dmat_part, "train")]
    #     hard_code: cpu_params cannot be an input, cause values are not homogeneous
    xgb_params = {
        "objective": "binary:logistic",
        "tree_method": "hist",
        "max_depth": 1,
        "nthread": 56,
        "eta": 0.1,
        "silent": 1,
        "subsample": 0.5,
        "colsample_bytree": 0.05,
        "eval_metric": "auc",
    }

    t0 = timer()
    model = xgboost.train(
        xgb_params,
        dtrain=training_dmat_part,
        num_boost_round=10000,
        evals=watchlist,
        early_stopping_rounds=30,
        maximize=True,
        verbose_eval=1000,
    )
    ml_times["t_train"] = timer() - t0

    t0 = timer()
    yp = pd.DataFrame({"preds": model.predict(testing_dmat_part)})
    ml_times["t_inference"] = timer() - t0

    ml_scores["mse"] = mse(y_test=y_test, y_pred=yp)
    ml_scores["cod"] = cod(y_test=y_test, y_pred=yp)

    ml_times["t_ml"] += ml_times["t_train"] + ml_times["t_inference"]

    return ml_scores, ml_times

In [10]:
@workflow
def santander_ml_wf(
    filename: str = DATASET_PATH,
    columns_names: typing.List[str] = COLUMNS_NAMES,
    columns_types: typing.List[str] = COLUMNS_TYPES,
    etl_keys: typing.List[str] = ETL_KEYS,
    target: str = "target",
    ml_keys: typing.List[str] = ML_KEYS,
    ml_score_keys: typing.List[str] = ML_SCORE_KEYS,
) -> (typing.Dict[str, float], typing.Dict[str, float]):
    df, etl_times = etl_pandas(
        filename=filename,
        columns_names=columns_names,
        columns_types=columns_types,
        etl_keys=etl_keys,
    )
    return ml(ml_data=df, target=target, ml_keys=ml_keys, ml_score_keys=ml_score_keys)

## <a id='execution_cell_2'>Workflow execution - more detalized task division (subworkflow inclusion)</a>

In [11]:
%%time

santander_ml_wf()

[18:11:27] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-auc:0.53415	train-auc:0.52877
[1000]	eval-auc:0.89019	train-auc:0.90135
[2000]	eval-auc:0.90305	train-auc:0.91705
[3000]	eval-auc:0.90846	train-auc:0.92279
[4000]	eval-auc:0.91038	train-auc:0.92592
[4348]	eval-auc:0.91127	train-auc:0.92682
CPU times: user 2h 3min 20s, sys: 1min 30s, total: 2h 4min 50s
Wall time: 4min 21s


DefaultNamedTupleOutput(o0={'mse_mean': 0.0, 'cod_mean': 0.0, 'mse_dev': 0.0, 'mse': preds    NaN
target   NaN
dtype: float64, 'cod': preds     NaN
target    1.0
dtype: float64}, o1={'t_train_test_split': 0.00010265223681926727, 't_dmatrix': 1.907059057150036, 't_training': 0.0, 't_infer': 0.0, 't_ml': 138.34493321506307, 't_train': 138.32563289115205, 't_inference': 0.01930032391101122})